In [43]:
import pandas as pd
import numpy as np
import glob
import os
import swifter

from tqdm import tqdm
from dataprep.eda import create_report

from utility_functions import get_year_from_filename

In [52]:
file_names = []
for f in glob.glob(os.path.join("data", "keyframes", '**\\*.csv'), recursive=True):
    file_names.append(f)

for f in glob.glob(os.path.join("data", "keyframes", '**\\*.xlsx'), recursive=True):
    file_names.append(f)

print("Identified {} .csv and .xlsx files...".format(len(file_names)))

Identified 94 .csv and .xlsx files...


In [53]:
file_names

['data\\keyframes\\2017\\Keyframes\\Body10_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body11_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body12_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body13_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body14_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body18_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body22_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body26_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body27_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body28_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body57_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body59_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body60_keyframes.csv',
 'data\\keyframes\\2017\\Keyframes\\Body61_keyframes.csv',
 'data\\keyframes\\2019\\Keyframes\\Body11_keyframes.csv',
 'data\\keyframes\\2019\\Keyframes\\Body12_keyframes.csv',
 'data\\keyframes\\2019\\Keyframes\\Body13_keyframes.csv

In [54]:
df = pd.DataFrame(columns=['bodyNumber', 'trial', 'year', 'T0', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'BR', 'BV', 'ORWF'])
# BEFORE RUNNING: Go to file data\keyframes\2021_A\Keyframes\Body69_keyframes.csv and delete the final comma in line 15
for file_name in tqdm(file_names):
    year = get_year_from_filename(file_name)
    if str.endswith(file_name, 'csv'):
        temp = pd.read_csv(file_name, sep=';|,', engine='python')
    else:
        temp = pd.read_excel(file_name)
    # add year
    temp.loc[:, 'year'] = year
    # remove empty, unnamed columns
    temp = temp.loc[:, ~temp.columns.str.contains('^Unnamed')]
    # remove filename column present only in 2021_B folder
    temp = temp.loc[:, ~temp.columns.str.contains('filename')]
    # remove white spaces from column names
    temp.columns = temp.columns.str.replace(' ', '')
    # append temp to main df
    df = pd.concat([df, temp], axis=0)
    if len(df.columns) != 13:
        print("There is a problem with the formatting of file {}. Please correct it before proceeding.".format(file_name))
        print(df.tail())
        break

100%|██████████| 94/94 [00:00<00:00, 186.20it/s]


In [55]:
print("*********** Dataframe Content: ***********\n")
print(df.head())

*********** Dataframe Content: ***********

  bodyNumber trial  year   T0   T1   T2   T3   T4   T5   T6   BR   BV ORWF
0         10     1  2017  NaN  NaN  350  532  601  655  790  NaN  NaN  NaN
1         10     2  2017  NaN  NaN  310  574  595  649  840  NaN  NaN  NaN
2         10     3  2017  NaN  NaN  269  533  600  654  792  NaN  NaN  NaN
3         10     4  2017  NaN  NaN  271  535  557  611  790  NaN  NaN  NaN
4         10     5  2017  NaN  NaN  214  478  533  587  815  NaN  NaN  NaN


In [56]:
print("*********** Dataframe Description: ***********\n")
print(df.describe())

*********** Dataframe Description: ***********

        bodyNumber  trial  year   T0   T1      T2      T3      T4      T5  \
count         1647   1647  1647  335  335  1494.0  1494.0  1645.0  1645.0   
unique          75     28     4   26  265   786.0   914.0   991.0   980.0   
top             13      6  2019   50  750     0.0    -1.0    -1.0    -1.0   
freq            55     94   881   87    4   128.0    32.0    32.0    32.0   

            T6   BR    BV  ORWF  
count   1542.0   67    40    27  
unique   939.0   66    39    25  
top       -1.0  741  1829   734  
freq      32.0    2     2     2  


In [57]:
print("*********** Dataframe Null Values per Column: ***********\n")
df.isna().sum()

*********** Dataframe Null Values per Column: ***********



bodyNumber       0
trial            0
year             0
T0            1312
T1            1312
T2             153
T3             153
T4               2
T5               2
T6             105
BR            1580
BV            1607
ORWF          1620
dtype: int64

In [59]:
df.to_csv(os.path.join("data", "keyframes.csv"))

In [18]:
report = create_report(df, title='Kinetic Analysis Keyframes').show_browser()

  0%|          | 0/2602 [00:00<?, ?it/s]

C:\Users\syfan\PycharmProjects\Drag-flik-RAIS-Hackathon\venv\lib\site-packages\dask\core.py:119: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


## Merge keyframes to original dataframe

In [44]:
print("Reading mvnx data...")
mvnx = pd.read_pickle(os.path.join("data", "mvnx_merged_data_train_validation_test_preprocessed.pkl"))
mvnx = mvnx.drop_duplicates()
mvnx.time = mvnx.time.astype(int)
print("Creating key...")
mvnx["key"] = mvnx['year'].astype(str) + '_' + mvnx['id'].astype(str) + '_' + mvnx['sample'].astype(str)
mvnx.head()

Reading mvnx data...
Creating key...


,Pelvis_acc_0,Pelvis_acc_1,Pelvis_acc_2,Pelvis_angular_acc_0,Pelvis_angular_acc_1,Pelvis_angular_acc_2,Pelvis_vel_0,Pelvis_vel_1,Pelvis_vel_2,Pelvis_angular_vel_0,...,LeftFoot_ori_3,LeftFoot_pos_0,LeftFoot_pos_1,LeftFoot_pos_2,time,year,id,sample,speed,key
0,0.065029,0.060930,-0.063273,0.848845,-0.554506,-1.050589,-0.069628,0.014435,0.004669,0.060686,...,-0.428791,8.701041,11.541834,0.171602,0,2017,1,1,65.0,2017_1_1
1,0.124831,-0.110555,-0.033831,-0.365366,-0.883694,-0.479441,-0.068602,0.014635,0.004517,0.059164,...,-0.428764,8.700834,11.541925,0.171608,4,2017,1,1,65.0,2017_1_1
2,0.149079,-0.071584,-0.092055,0.063539,-0.969149,-0.945796,-0.067396,0.014898,0.004128,0.059429,...,-0.428748,8.700640,11.542024,0.171618,9,2017,1,1,65.0,2017_1_1
3,0.071367,-0.329402,-0.074564,-1.390745,-0.642128,-0.478393,-0.066487,0.014024,0.003817,0.053634,...,-0.428726,8.700454,11.542136,0.171630,13,2017,1,1,65.0,2017_1_1
4,0.150359,-0.121912,-0.028015,-0.380801,-1.111007,-0.887547,-0.065267,0.013861,0.003700,0.052047,...,-0.428685,8.700278,11.542247,0.171644,17,2017,1,1,65.0,2017_1_1


In [45]:
# keep only the columns without null values for segmentation
print("Reading keyframe data...")
df = pd.read_csv(os.path.join("data", "keyframes.csv"), index_col=[0])
# df = df[['bodyNumber', 'trial', 'year', 'T4', 'T5']]
print("Creating key...")
df["key"] = df['year'].astype(str) + '_' + df['bodyNumber'].astype(str) + '_' + df['trial'].astype(str)
df.head()

Reading keyframe data...
Creating key...


,bodyNumber,trial,year,T0,T1,T2,T3,T4,T5,T6,BR,BV,ORWF,key
0,10,1,2017,NaN,NaN,350.0,532.0,601.0,655.0,790.0,NaN,NaN,NaN,2017_10_1
1,10,2,2017,NaN,NaN,310.0,574.0,595.0,649.0,840.0,NaN,NaN,NaN,2017_10_2
2,10,3,2017,NaN,NaN,269.0,533.0,600.0,654.0,792.0,NaN,NaN,NaN,2017_10_3
3,10,4,2017,NaN,NaN,271.0,535.0,557.0,611.0,790.0,NaN,NaN,NaN,2017_10_4
4,10,5,2017,NaN,NaN,214.0,478.0,533.0,587.0,815.0,NaN,NaN,NaN,2017_10_5


In [46]:
# merged = mvnx.merge(df, how='left', left_on=['year', 'id', 'sample'], right_on=['year', 'bodyNumber', 'trial'])
# merged.head()

In [47]:
# adding Ti in the mvnx data
mvnx[['segment']] = np.NaN
mvnx

,Pelvis_acc_0,Pelvis_acc_1,Pelvis_acc_2,Pelvis_angular_acc_0,Pelvis_angular_acc_1,Pelvis_angular_acc_2,Pelvis_vel_0,Pelvis_vel_1,Pelvis_vel_2,Pelvis_angular_vel_0,...,LeftFoot_pos_0,LeftFoot_pos_1,LeftFoot_pos_2,time,year,id,sample,speed,key,segment
0,0.065029,0.060930,-0.063273,0.848845,-0.554506,-1.050589,-0.069628,0.014435,0.004669,0.060686,...,8.701041,11.541834,0.171602,0,2017,1,1,65.0,2017_1_1,NaN
1,0.124831,-0.110555,-0.033831,-0.365366,-0.883694,-0.479441,-0.068602,0.014635,0.004517,0.059164,...,8.700834,11.541925,0.171608,4,2017,1,1,65.0,2017_1_1,NaN
2,0.149079,-0.071584,-0.092055,0.063539,-0.969149,-0.945796,-0.067396,0.014898,0.004128,0.059429,...,8.700640,11.542024,0.171618,9,2017,1,1,65.0,2017_1_1,NaN
3,0.071367,-0.329402,-0.074564,-1.390745,-0.642128,-0.478393,-0.066487,0.014024,0.003817,0.053634,...,8.700454,11.542136,0.171630,13,2017,1,1,65.0,2017_1_1,NaN
4,0.150359,-0.121912,-0.028015,-0.380801,-1.111007,-0.887547,-0.065267,0.013861,0.003700,0.052047,...,8.700278,11.542247,0.171644,17,2017,1,1,65.0,2017_1_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1818484,-0.678712,-0.744590,-1.186564,-3.637199,-0.377002,4.884401,0.055096,0.064242,0.032775,-0.310567,...,6.571525,0.447844,0.226406,3234,2021,83,2,80.0,2021_83_2,NaN
1818485,-0.940335,-1.197655,-1.366915,-7.152955,0.081270,5.653244,0.051436,0.061982,0.027549,-0.340371,...,6.569550,0.451821,0.224835,3238,2021,83,2,80.0,2021_83_2,NaN
1818486,-0.831967,-1.163168,-1.292596,-5.512093,-0.151179,3.237055,0.047789,0.059788,0.022355,-0.363338,...,6.567587,0.455787,0.223269,3242,2021,83,2,80.0,2021_83_2,NaN
1818487,-0.617368,-0.357365,-1.066019,-1.425752,-2.776401,5.538847,0.044178,0.057759,0.017240,-0.369279,...,6.565653,0.459721,0.221714,3246,2021,83,2,80.0,2021_83_2,NaN


In [48]:
for row in tqdm(df.iterrows()):
    # get data from keyframes row
    key = row[1].key
    T2 = row[1].T2
    T3 = row[1].T3
    T4 = row[1].T4
    T5 = row[1].T5
    T6 = row[1].T6
    # adding True (1) to the rows referring to each time segment
    mvnx.loc[(mvnx["key"]==key) & (mvnx["time"] >= T2) & (mvnx["time"] < T3), "segment"] = "T2"
    mvnx.loc[(mvnx["key"]==key) & (mvnx["time"] >= T3) & (mvnx["time"] < T4), "segment"] = "T3"
    mvnx.loc[(mvnx["key"]==key) & (mvnx["time"] >= T4) & (mvnx["time"] < T5), "segment"] = "T4"
    mvnx.loc[(mvnx["key"]==key) & (mvnx["time"] >= T5) & (mvnx["time"] < T6), "segment"] = "T5"
mvnx.head()

1647it [04:44,  5.78it/s]


,Pelvis_acc_0,Pelvis_acc_1,Pelvis_acc_2,Pelvis_angular_acc_0,Pelvis_angular_acc_1,Pelvis_angular_acc_2,Pelvis_vel_0,Pelvis_vel_1,Pelvis_vel_2,Pelvis_angular_vel_0,...,LeftFoot_pos_0,LeftFoot_pos_1,LeftFoot_pos_2,time,year,id,sample,speed,key,segment
0,0.065029,0.060930,-0.063273,0.848845,-0.554506,-1.050589,-0.069628,0.014435,0.004669,0.060686,...,8.701041,11.541834,0.171602,0,2017,1,1,65.0,2017_1_1,NaN
1,0.124831,-0.110555,-0.033831,-0.365366,-0.883694,-0.479441,-0.068602,0.014635,0.004517,0.059164,...,8.700834,11.541925,0.171608,4,2017,1,1,65.0,2017_1_1,NaN
2,0.149079,-0.071584,-0.092055,0.063539,-0.969149,-0.945796,-0.067396,0.014898,0.004128,0.059429,...,8.700640,11.542024,0.171618,9,2017,1,1,65.0,2017_1_1,NaN
3,0.071367,-0.329402,-0.074564,-1.390745,-0.642128,-0.478393,-0.066487,0.014024,0.003817,0.053634,...,8.700454,11.542136,0.171630,13,2017,1,1,65.0,2017_1_1,NaN
4,0.150359,-0.121912,-0.028015,-0.380801,-1.111007,-0.887547,-0.065267,0.013861,0.003700,0.052047,...,8.700278,11.542247,0.171644,17,2017,1,1,65.0,2017_1_1,NaN


In [49]:
# Testing if it works
# correct T2=350, T3=532, T4=601, T5=655, T6=790
mvnx.loc[mvnx.key=="2017_10_1"]

,Pelvis_acc_0,Pelvis_acc_1,Pelvis_acc_2,Pelvis_angular_acc_0,Pelvis_angular_acc_1,Pelvis_angular_acc_2,Pelvis_vel_0,Pelvis_vel_1,Pelvis_vel_2,Pelvis_angular_vel_0,...,LeftFoot_pos_0,LeftFoot_pos_1,LeftFoot_pos_2,time,year,id,sample,speed,key,segment
2692,0.100720,-0.158526,-0.072276,-1.132179,-0.265471,-0.409607,0.000684,0.017791,-0.005597,0.001888,...,1.619714,9.163127,0.162508,0,2017,10,1,79.0,2017_10_1,NaN
2693,0.123224,0.046644,-0.091935,-0.158352,0.101824,0.032851,0.001245,0.017702,-0.005931,0.001228,...,1.619729,9.163137,0.162488,4,2017,10,1,79.0,2017_10_1,NaN
2694,0.040653,-0.079366,-0.067010,-0.894743,0.377400,-0.026166,0.001366,0.017094,-0.006136,-0.002500,...,1.619737,9.163137,0.162468,8,2017,10,1,79.0,2017_10_1,NaN
2695,0.058090,0.029313,-0.060487,-0.070511,-0.122347,-0.773893,0.001459,0.016905,-0.006294,-0.002794,...,1.619737,9.163125,0.162448,13,2017,10,1,79.0,2017_10_1,NaN
2696,0.037328,-0.004200,-0.032659,-0.110013,0.046052,-0.073637,0.001461,0.016572,-0.006323,-0.003252,...,1.619735,9.163112,0.162429,17,2017,10,1,79.0,2017_10_1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5029,-0.069188,0.065409,0.989158,2.790476,-1.601495,-5.520876,0.926644,-0.391370,0.395236,-0.043358,...,11.230515,5.828831,0.176877,4999,2017,10,1,79.0,2017_10_1,NaN
5030,0.127856,0.445422,0.586566,4.601451,-1.468475,-1.902815,0.935452,-0.387229,0.392885,-0.024186,...,11.231894,5.828888,0.177889,5003,2017,10,1,79.0,2017_10_1,NaN
5031,-0.065614,0.680388,-0.060183,6.079559,0.043252,-1.316606,0.944233,-0.383161,0.388151,0.001146,...,11.233307,5.828946,0.178902,5007,2017,10,1,79.0,2017_10_1,NaN
5032,0.171911,0.491820,-0.146881,4.178833,-1.651495,-0.914933,0.953866,-0.380842,0.384128,0.018558,...,11.234719,5.829018,0.180074,5011,2017,10,1,79.0,2017_10_1,NaN


In [50]:
har = mvnx.loc[mvnx.segment.notnull()]
print("Original shape: {} - HAR shape: {}".format(mvnx.shape[0], har.shape[0]))

Original shape: 1057637 - HAR shape: 91715


In [51]:
har.head()

,Pelvis_acc_0,Pelvis_acc_1,Pelvis_acc_2,Pelvis_angular_acc_0,Pelvis_angular_acc_1,Pelvis_angular_acc_2,Pelvis_vel_0,Pelvis_vel_1,Pelvis_vel_2,Pelvis_angular_vel_0,...,LeftFoot_pos_0,LeftFoot_pos_1,LeftFoot_pos_2,time,year,id,sample,speed,key,segment
2776,-0.054363,0.056517,-0.053088,0.420520,0.209671,-0.530950,0.007970,-0.003923,-0.001043,-0.000674,...,1.618867,9.162230,0.161790,350,2017,10,1,79.0,2017_10_1,T2
2777,-0.107037,0.053799,-0.122576,-0.002736,0.640158,0.677067,0.007299,-0.004135,-0.001530,-0.000686,...,1.618847,9.162230,0.161792,354,2017,10,1,79.0,2017_10_1,T2
2778,0.144353,-0.014925,-0.004969,-0.029263,-1.064792,0.387637,0.007754,-0.004556,-0.001520,-0.000808,...,1.618827,9.162235,0.161795,359,2017,10,1,79.0,2017_10_1,T2
2779,-0.030959,-0.105743,-0.107790,-1.158075,0.311378,0.144316,0.007494,-0.005328,-0.001936,-0.005633,...,1.618806,9.162238,0.161799,363,2017,10,1,79.0,2017_10_1,T2
2780,0.059421,-0.083124,0.010181,-0.362501,-0.402047,-0.376975,0.007641,-0.005942,-0.001857,-0.007143,...,1.618786,9.162240,0.161802,367,2017,10,1,79.0,2017_10_1,T2


In [54]:
mvnx.to_pickle(os.path.join("data", "mvnx_merged_data_train_validation_test_preprocessed_har.pkl"))

In [55]:
har.to_pickle(os.path.join("data", "mvnx_merged_data_train_validation_test_preprocessed_har_without_null.pkl"))